In [8]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.0 MB/s  0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.3
    Uninstalling pip-25.3:
      Successfully uninstalled pip-25.3
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install pandas


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [6]:
#installing sklarn
!pip install scikit-learn

  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 16.7 MB/s  0:00:00 eta 0:00:01
Using cached joblib-1.5.3-py3-none-any.whl (309 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 17.1 MB/s  0:00:01 eta 0:00:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
import pickle #Used to save the trained TF-IDF vectorizer to disk.
from sklearn.feature_extraction.text import TfidfVectorizer # convert text into numerical features.
from sklearn.model_selection import train_test_split # split dataset into training and testing sets.
import os

In [12]:
def create_features(input_file, models_dir):
    print(f"Loading {input_file}...")
    df = pd.read_csv(input_file)

    X = df['cleaned_text'].fillna('')
    y = df['risk_level']


    # Creating TF-IDF Vectorizer
    vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),  #1-word phrases (unigrams) and 2-word phrases (bigrams)
    max_features=5000, #Limits vocabulary size to top 5000 most important words/phrases. 
    stop_words='english') #Removes common words

    X_tfidf = vectorizer.fit_transform(X) # Converts text to numerical features.

    print(f"Vocabulary size: {len(vectorizer.vocabulary_)}")

    X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42, stratify=y)
    #! stratify=y : The distribution of High / Medium / Low is the same in both train and test sets.

    os.makedirs(models_dir, exist_ok=True) #Creates models/ directory if it doesn't exist.
    vec_path = os.path.join(models_dir, 'tfidf_vectorizer.pkl') #Builds save path.

    with open(vec_path, 'wb') as f: # 'wb' = write binary.
        pickle.dump(vectorizer, f) # Saves the trained TF-IDF vectorizer to disk.
        # why save :- we must use the same vocabulary when predicting new contracts.

    return X_train, X_test, y_train, y_test

if __name__ == "__main__":
    try:
        base_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
    except NameError:
        base_dir = os.path.dirname(os.getcwd()) #Gets current file location
    input_path = os.path.join(base_dir, "data", "processed", "processed_contracts.csv")
    models_dir = os.path.join(base_dir, "models") # Directory to save trained models and vectorizers.
    
    create_features(input_path, models_dir)

Loading /Users/ash/CascadeProjects/projects..../contrack_risk_analyser/data/processed/processed_contracts.csv...
Vocabulary size: 5000
